# Seznam - ...

...

Summary:

- Prediction type: __Regression model__
- Domain: __Sales__
- Prediction target: __Salaries__ 
- Population size: __23111__

_Author: Dr. Patrick Urbanke_

# Background

...

The dataset has been downloaded from the [CTU Prague relational learning repository](https://relational.fit.cvut.cz/dataset/VOC) (Motl and Schulte, 2015).

We will benchmark [getML](https://www.getml.com) 's feature learning algorithms against [featuretools](https://www.featuretools.com), an open-source implementation of the propositionalization algorithm, similar to getML's FastProp.

### A web frontend for getML

The getML monitor is a frontend built to support your work with getML. The getML monitor displays information such as the imported data frames, trained pipelines and allows easy data and feature exploration. You can launch the getML monitor [here](http://localhost:1709).

### Where is this running?

Your getML live session is running inside a docker container on [mybinder.org](https://mybinder.org/), a service built by the Jupyter community and funded by Google Cloud, OVH, GESIS Notebooks and the Turing Institute. As it is a free service, this session will shut down after 10 minutes of inactivity.

# Analysis

Let's get started with the analysis and set up your session:

In [1]:
import copy
import os
from urllib import request

import numpy as np
import pandas as pd
from IPython.display import Image
import matplotlib.pyplot as plt
plt.style.use('seaborn')
%matplotlib inline  

import featuretools
import getml

getml.engine.set_project('seznam')



Connected to project 'seznam'


## 1. Loading data

### 1.1 Download from source

We begin by downloading the data:

In [2]:
conn = getml.database.connect_mariadb(
    host="relational.fit.cvut.cz",
    dbname="Seznam",
    port=3306,
    user="guest",
    password="relational"
)

conn

Connection(conn_id='default',
           dbname='Seznam',
           dialect='mysql',
           host='relational.fit.cvut.cz',
           port=3306)

In [3]:
def load_if_needed(name):
    """
    Loads the data from the relational learning
    repository, if the data frame has not already
    been loaded.
    """
    if not getml.data.exists(name):
        data_frame = getml.data.DataFrame.from_db(
            name=name,
            table_name=name,
            conn=conn
        )
        data_frame.save()
    else:
        data_frame = getml.data.load_data_frame(name)
    return data_frame

In [4]:
dobito = load_if_needed("dobito")
probehnuto = load_if_needed("probehnuto")
probehnuto_mimo_penezenku = load_if_needed("probehnuto_mimo_penezenku")

In [5]:
dobito

AttributeError: '_DataFrameFormatColumn' object has no attribute 'precision'

AttributeError: '_DataFrameFormatColumn' object has no attribute 'precision'

In [6]:
probehnuto

AttributeError: '_DataFrameFormatColumn' object has no attribute 'precision'

AttributeError: '_DataFrameFormatColumn' object has no attribute 'precision'

In [7]:
probehnuto_mimo_penezenku

AttributeError: '_DataFrameFormatColumn' object has no attribute 'precision'

AttributeError: '_DataFrameFormatColumn' object has no attribute 'precision'

### 1.2 Prepare data for getML

getML requires that we define *roles* for each of the columns.

In [8]:
dobito.set_role("client_id", getml.data.roles.join_key)
dobito.set_role("month_year_datum_transakce", getml.data.roles.time_stamp)
dobito.set_role("sluzba", getml.data.roles.categorical)
dobito.set_role("kc_dobito", getml.data.roles.numerical)

dobito.set_unit("sluzba", "service")

dobito

AttributeError: '_DataFrameFormatColumn' object has no attribute 'precision'

AttributeError: '_DataFrameFormatColumn' object has no attribute 'precision'

In [9]:
probehnuto.set_role("client_id", getml.data.roles.join_key)
probehnuto.set_role("month_year_datum_transakce", getml.data.roles.time_stamp)
probehnuto.set_role("sluzba", getml.data.roles.categorical)
probehnuto.set_role("kc_proklikano", getml.data.roles.target)

probehnuto.set_unit("sluzba", "service")

probehnuto

AttributeError: '_DataFrameFormatColumn' object has no attribute 'precision'

AttributeError: '_DataFrameFormatColumn' object has no attribute 'precision'

In [10]:
probehnuto_mimo_penezenku.set_role("client_id", getml.data.roles.join_key)
probehnuto_mimo_penezenku.set_role("Month/Year", getml.data.roles.time_stamp)

probehnuto_mimo_penezenku

AttributeError: '_DataFrameFormatColumn' object has no attribute 'precision'

AttributeError: '_DataFrameFormatColumn' object has no attribute 'precision'

In [11]:
split = getml.data.split.random(train=0.8, test=0.2)
split

,
0,train
1,train
2,train
3,test
4,train
,...


## 2. Predictive modeling

We loaded the data and defined the roles and units. Next, we create a getML pipeline for relational learning.

### 2.1 Define relational model

In [12]:
star_schema = getml.data.StarSchema(population=probehnuto, alias="population", split=split)

star_schema.join(
    probehnuto,
    on="client_id",
    time_stamps="month_year_datum_transakce",
    lagged_targets=True,
    horizon=getml.data.time.days(1),
)

star_schema.join(
    dobito,
    on="client_id",
    time_stamps="month_year_datum_transakce",
)

star_schema.join(
    probehnuto_mimo_penezenku,
    on="client_id", 
    time_stamps=("month_year_datum_transakce",  "Month/Year"),
)

star_schema

### 2.2 getML pipeline

<!-- #### 2.1.1  -->
__Set-up the feature learner & predictor__

We use the relboost algorithms for this problem. Because of the large number of keywords, we regularize the model a bit by requiring a minimum support for the keywords (`min_num_samples`).

In [ ]:
mapping = getml.preprocessors.Mapping()

fast_prop = getml.feature_learning.FastProp(
    aggregation=getml.feature_learning.FastProp.agg_sets.All,
    loss_function=getml.feature_learning.loss_functions.SquareLoss,
    num_threads=1,    
    sampling_factor=0.1,
)

predictor = getml.predictors.XGBoostRegressor(n_jobs=1)

__Build the pipeline__

In [ ]:
pipe1 = getml.pipeline.Pipeline(
    tags=['fast_prop'],
    data_model=star_schema.data_model,
    preprocessors=[mapping],
    feature_learners=[fast_prop],
    predictors=[predictor],
    include_categorical=True,
)

pipe1

### 2.3 Model training

In [ ]:
pipe1.check(star_schema.train)

In [ ]:
pipe1.fit(star_schema.train)

### 2.4 Model evaluation

In [ ]:
pipe1.score(star_schema.test)

### 2.5 featuretools

In [13]:
include = (getml.data.random() < 0.25)
include

AttributeError: 'BooleanColumnView' object has no attribute 'subroles'

AttributeError: 'BooleanColumnView' object has no attribute 'subroles'

In [15]:
population_train_pd = star_schema.train.population[include].to_pandas()
population_test_pd = star_schema.test.population.to_pandas()

In [16]:
population_train_pd["id"] = population_train_pd.index
population_test_pd["id"] = population_test_pd.index

In [17]:
probehnuto_pd = probehnuto.drop(probehnuto.unused_names).to_pandas()
dobito_pd = dobito.drop(dobito.unused_names).to_pandas()
probehnuto_mimo_penezenku_pd = probehnuto_mimo_penezenku.drop(probehnuto_mimo_penezenku.unused_names).to_pandas()

In [18]:
def prepare_peripheral(peripheral_pd, train_or_test):
    """
    Helper function that imitates the behavior of 
    the data model defined above.
    """
    peripheral_new = peripheral_pd.merge(
        train_or_test[["id", "client_id", "month_year_datum_transakce"]],
        on="client_id"
    )

    peripheral_new = peripheral_new[
        peripheral_new["month_year_datum_transakce_x"] < peripheral_new["month_year_datum_transakce_y"]
    ]

    del peripheral_new["month_year_datum_transakce_y"]
    del peripheral_new["client_id"]

    return peripheral_new.rename({"month_year_datum_transakce_y": "month_year_datum_transakce"})

In [19]:
def prepare_probehnuto_mimo_penezenku(peripheral_pd, train_or_test):
    """
    Helper function that imitates the behavior of 
    the data model defined above.
    """
    peripheral_new = peripheral_pd.merge(
        train_or_test[["id", "client_id", "month_year_datum_transakce"]],
        on="client_id"
    )

    peripheral_new = peripheral_new[
        peripheral_new["Month/Year"] < peripheral_new["month_year_datum_transakce"]
    ]

    del peripheral_new["month_year_datum_transakce"]
    del peripheral_new["client_id"]

    return peripheral_new

In [20]:
dobito_train_pd = prepare_peripheral(dobito_pd, population_train_pd)
dobito_test_pd = prepare_peripheral(dobito_pd, population_test_pd)
dobito_train_pd

,month_year_datum_transakce_x,sluzba,kc_dobito,id
0,2012-10-01,c,1045.62,2127
1,2012-10-01,c,1045.62,17709
2,2012-10-01,c,1045.62,50363
3,2015-03-01,a,25214.20,2127
4,2015-03-01,a,25214.20,17709
...,...,...,...,...
4462025,2012-08-01,g,37680.00,217660
4462026,2012-08-01,g,37680.00,235321
4462027,2012-08-01,g,37680.00,253661
4462028,2012-08-01,g,37680.00,264765


In [21]:
probehnuto_train_pd = prepare_peripheral(probehnuto_pd, population_train_pd)
probehnuto_test_pd = prepare_peripheral(probehnuto_pd, population_test_pd)
probehnuto_train_pd

,month_year_datum_transakce_x,kc_proklikano,sluzba,id
1,2013-06-01,-31.40,c,281262
4,2013-06-01,-31.40,c,288356
6,2013-06-01,-31.40,c,289265
7,2013-06-01,-31.40,c,289267
10,2013-06-01,-31.40,c,290759
...,...,...,...,...
11186670,2012-08-01,948.28,NULL,291919
11186671,2012-09-01,979.68,e,291919
11186673,2012-08-01,948.28,NULL,291848
11186674,2012-08-01,87.92,e,291848


In [22]:
probehnuto_mimo_penezenku_train_pd = prepare_probehnuto_mimo_penezenku(probehnuto_mimo_penezenku_pd, population_train_pd)
probehnuto_mimo_penezenku_test_pd = prepare_probehnuto_mimo_penezenku(probehnuto_mimo_penezenku_pd, population_test_pd)
probehnuto_mimo_penezenku_train_pd

,Month/Year,id
0,2012-08-01,269301
8,2012-08-01,9204
9,2012-08-01,23838
10,2012-08-01,24471
11,2012-08-01,45868
...,...,...
3568048,2015-09-01,160015
3568050,2015-09-01,19
3568051,2015-09-01,1565
3568053,2015-09-01,151283


In [23]:
del population_train_pd["client_id"]
del population_test_pd["client_id"]

In [24]:
population_train_pd

,month_year_datum_transakce,kc_proklikano,sluzba,id
0,2013-06-01,-31.40,c,0
1,2015-10-01,725.34,h,1
2,2015-10-01,8550.22,h,2
3,2015-10-01,2408.38,h,3
4,2015-10-01,1893.42,h,4
...,...,...,...,...
292153,2015-03-01,153.86,NULL,292153
292154,2015-05-01,153.86,NULL,292154
292155,2015-06-01,13545.96,NULL,292155
292156,2015-06-01,153.86,NULL,292156


In [25]:
entities_train = {
    "population" : (population_train_pd, "id"),
    "dobito": (dobito_train_pd, "index"),
    "probehnuto": (probehnuto_train_pd, "index"),
    "probehnuto_mimo_penezenku": (probehnuto_mimo_penezenku_train_pd, "index"),
}

In [26]:
entities_test = {
    "population" : (population_test_pd, "id"),
    "dobito": (dobito_test_pd, "index"),
    "probehnuto": (probehnuto_test_pd, "index"),
    "probehnuto_mimo_penezenku": (probehnuto_mimo_penezenku_test_pd, "index"),
}

In [27]:
relationships = [
    ("population", "id", "dobito", "id"),
    ("population", "id", "probehnuto", "id"),
    ("population", "id", "probehnuto_mimo_penezenku", "id"),
]

In [ ]:
featuretools_train_pd = featuretools.dfs(
    entities=entities_train,
    relationships=relationships,
    target_entity="population")[0]

/home/patrick/.local/lib/python3.9/site-packages/featuretools/entityset/entity.py:462: UserWarning: index index not found in dataframe, creating new integer column
  warnings.warn("index {} not found in dataframe, creating new "


In [ ]:
featuretools_test_pd = featuretools.dfs(
    entities=entities_test,
    relationships=relationships,
    target_entity="population")[0]

In [ ]:
featuretools_train = getml.data.DataFrame.from_pandas(featuretools_train_pd, "featuretools_train")
featuretools_test = getml.data.DataFrame.from_pandas(featuretools_test_pd, "featuretools_test")

In [ ]:
featuretools_train.set_role("salary", getml.data.roles.target)
featuretools_train.set_role(featuretools_train.unused_float_names, getml.data.roles.numerical)
featuretools_train.set_role(featuretools_train.unused_string_names, getml.data.roles.categorical)

featuretools_train

In [ ]:
featuretools_test.set_role("salary", getml.data.roles.target)
featuretools_test.set_role(featuretools_test.unused_float_names, getml.data.roles.numerical)
featuretools_test.set_role(featuretools_test.unused_string_names, getml.data.roles.categorical)

featuretools_test

We train an untuned XGBoostRegressor on top of featuretools' features, just like we have done for getML's features.

Since some of featuretools features are categorical, we allow the pipeline to include these features as well. Other features contain NaN values, which is why we also apply getML's Imputation preprocessor.

In [ ]:
data_model = getml.data.DataModel("population")

In [ ]:
imputation = getml.preprocessors.Imputation()

predictor = getml.predictors.XGBoostRegressor(n_jobs=1)

pipe2 = getml.pipeline.Pipeline(
    tags=['featuretools'],
    data_model=data_model,
    preprocessors=[imputation],
    predictors=[predictor],
    include_categorical=True,
)

pipe2

In [ ]:
pipe2.fit(featuretools_train)

In [ ]:
pipe2.score(featuretools_test)

### 2.7 Discussion

For a more convenient overview, we summarize our results into a table.

Name                 | R-squared  | RMSE      | MAE
-------------------- | ---------- | ----------| ----
getML: FastProp      |     78.80% | 1,402,960 | 765,292
getML: Relboost      |     83.95% | 1,220,382 | 666,548
featuretools         |     77.97% | 1,431,516 | 769,939



## 3. Conclusion

...

## References

Motl, Jan, and Oliver Schulte. "The CTU prague relational learning repository." arXiv preprint arXiv:1511.03086 (2015).

# Next Steps

This tutorial benchmarked getML against academic state-of-the-art algorithms from relational learning literature and getML's qualities with respect to categorical data.

If you are interested in further real-world applications of getML, head back to the [notebook overview](welcome.md) and choose one of the remaining examples.

Here is some additional material from our [documentation](https://docs.getml.com/latest/) if you want to learn more about getML:
* [Feature learning with Multirel](https://docs.getml.com/latest/user_guide/feature_engineering/feature_engineering.html#multirel)
* [Feature learning with Relboost](https://docs.getml.com/latest/user_guide/feature_engineering/feature_engineering.html#relboost)

# Get in contact

If you have any question schedule a [call with Alex](https://go.getml.com/meetings/alexander-uhlig/getml-demo), the co-founder of getML, or write us an [email](team@getml.com). Prefer a private demo of getML? Just contact us to make an appointment.